Benjamin Katz <br>NLP part 1<br>Due: Week 11<br> Due: April 17

A special thanks to the Gensim documentation as well as the original Word to Vector research paper for guiding me in this assignment

In [ ]:
import nltk
import pandas as pd

In [ ]:
#Download the data just as proffessor dave did in his colab
!pip install kaggle --upgrade
print ("writing kaggle")
!echo "{\"username\":\"benjykatz\",\"key\":\"572fb2bc0eb7cb591fc708326d55de5d\"}" > kaggle.json
!sudo mkdir -p ~/.kaggle
!sudo cp /content/kaggle.json ~/.kaggle/kaggle.json
print ("got kaggle json copied")
!chmod 600 /root/.kaggle/kaggle.json
!kaggle --version
!kaggle competitions download -c sentiment-analysis-on-movie-reviews

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
writing kaggle
got kaggle json copied
Kaggle API 1.5.13
  0% 0.00/1.90M [00:00<?, ?B/s]
100% 1.90M/1.90M [00:00<00:00, 145MB/s]


In [ ]:
#Unzip the data
!unzip sentiment-analysis-on-movie-reviews.zip

Archive:  sentiment-analysis-on-movie-reviews.zip
  inflating: sampleSubmission.csv    
  inflating: test.tsv.zip            
  inflating: train.tsv.zip           


In [ ]:
#turn the table into a pandas dataframe
train_df = pd.read_table('/content/train.tsv.zip', compression = 'zip')
test_df = pd.read_table('/content/test.tsv.zip', compression = 'zip')

In [ ]:
#peek at the data
train_df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


I tokenized the words by seperating around the spaces and converting all text to lowercase. There is no real difference between upper and lower case words so I want my model to look at them as though they are the same. Additionally, becasue I am doing a word to vector I need the context in which the word appears. Therefor I am only looking at entries that the phrase is at least 4 words. I get no context by looking at a single word by itself.

In [ ]:
#Create a list of sentences where each sentence is a list of words in the order they appear
list_of_sentences = []
#Go through the database looking at the phrase column
for phrase in train_df['Phrase']:
  #change the phrase to lowercase
  phrase = phrase.lower()
  #Only look at a phrase that is 4 words or longer
  if(len(phrase.split(" "))>3):
    #add the sentence to the list
    list_of_sentences.append(phrase.split(" "))
#look at a random sentence to confirm it is the way it is supposed to be.
print(list_of_sentences[500])

['the', ',', 'a', 'of', 'and', 'to', '.', "'s", 'in', 'is', 'that', 'it', 'as', 'with', 'for', 'its', 'film', 'an', 'movie', 'but', 'be', 'this', 'on', 'you', 'The', "n't", 'by', 'more', 'his', 'at', 'about', '--', 'than', 'from', 'not', 'or', 'one', 'have', 'are', 'has', 'all', 'like', "'", 'so', 'A', 'who', '-RRB-', 'story', 'out', 'into', '-LRB-', 'most', 'up', 'too', '...', '`', '``', 'good', 'characters', 'I', 'can', "''", 'much', 'their', 'if', 'what', 'does', 'just', 'your', 'will', 'some', 'no', 'time', 'comedy', 'even', 'been', 'way', 'which', 'make', 'It', 'funny', 'little', 'any', 'life', 'he', 'only', 'very', 'movies', 'was', 'do', 'us', 'they', 'her', 'enough', 'work', 'would', 'own', 'through', 'other', 'something', 'never', 'made', 'makes', 'we', 'bad', 'when', 'many', 'people', 'love', 'may', 'how', 'see', 'could', 'director', 'best', 'them', 'two', 'action', 'there', 'films', 'well', 'plot', 'character', 'without', 'being', 'audience', 'really', 'look', 'should', "'re"

Use the genism library to create these word vectors, the model uses skip-gram and CBOW models to create this vector. Essentially, it both tries to project meaning onto a vector using a non ordered frequency analysis(CBOW) while also ascribing meaning based on the meaning of the words around it(skip-gram).

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
#Create a model using genism Word2Vec with the sentences that we just got from our dataset and for each word make a vector of size 100
model = Word2Vec(sentences=list_of_sentences, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [ ]:
#A list of words that we will find the 10 most sililar words using our model
words_to_find_similar = ['jar-jar', 'exciting', 'sad', 'story', 'king', 'funny', 'actor', 'movie', 'college', 'simple', 'sex', 'hilarious', 'jesus', 'god', 'documentary', 'queen']

In [ ]:
for word in words_to_find_similar:
  print()
  print(word)
  #Using cosine distance find the 10 nearest vectors
  print(model.wv.most_similar(word, topn = 10))


jar-jar
[('binks', 0.9747676849365234), ('1973', 0.8869577050209045), ('webcast', 0.882804274559021), ('gadzooks', 0.8718612790107727), ('sever', 0.8590529561042786), ('ecks', 0.857142984867096), ('punk', 0.8546275496482849), ('clockstoppers', 0.8539853692054749), ('tango', 0.8423541188240051), ('novelty', 0.8371754288673401)]

exciting
[('institution', 0.8763965368270874), ('realistic', 0.8573500514030457), ('dramatically', 0.8464429974555969), ('inspiring', 0.8441445231437683), ('craftsmanship', 0.8437492847442627), ('wickedly', 0.8419249057769775), ('oddly', 0.8419027924537659), ('urges', 0.8416705131530762), ('purposeless', 0.8351890444755554), ('unsatisfying', 0.8343991637229919)]

sad
[('reflective', 0.8294448852539062), ('atrocious', 0.8113255500793457), ('vulgar', 0.8059430122375488), ('gritty', 0.7998274564743042), ('fabulous', 0.7957385182380676), ('sublimely', 0.7904833555221558), ('messy', 0.7901947498321533), ('pointless', 0.7885215282440186), ('raucously', 0.785626292228

Some of these results make a lot of sense while some seem more random. In the casew where the word likely shows up in many contexts the model does a better job at accurately predicting similar words. Words like movie, it corectly associates with film and picture. Hillarious with enrgetic and unpredictable. But words like college, which it likely does not see so often, are assocaited with van and hospital. King is associated with lion becasue of how often the movie lion king is mentioned in the dataset. In the real world the word king has words that are more related than lion, but because we are biased to a movie data set, king is closest to lion.<br> 
Often words that are complete oposites are similar. This makes sense becuase they would often appear in similar contexts in order to contrast. Also opposites are similar, men and women have a lot more in common than man and toast. 

In [ ]:
# Using cosine similarity, find the nearest word to the positive that is farthest from the negative
#Aka vector arithmetic

result = model.wv.most_similar(positive=['woman', 'king'], negative=['man'])
most_similar_key, similarity = result[0]  # look at the first match
print("king+woman-man = "+most_similar_key)

result = model.wv.most_similar(positive=['romantic', 'funny'])
most_similar_key, similarity = result[0]  # look at the first match
print("romantic+funny = "+most_similar_key)

result = model.wv.most_similar(positive=['boy', 'mature'])
most_similar_key, similarity = result[0]  # look at the first match
print("boy+mature = "+most_similar_key)

result = model.wv.most_similar(positive=['man', 'clever'])
most_similar_key, similarity = result[0]  # look at the first match
print("man+clever = "+most_similar_key)

result = model.wv.most_similar(positive=['woman'], negative = ['dumb'])
most_similar_key, similarity = result[0]  # look at the first match
print("woman-dumb = "+most_similar_key)#Yikes

result = model.wv.most_similar(positive=['woman'], negative = ['smart'])
most_similar_key, similarity = result[0]  # look at the first match
print("woman-smart = "+most_similar_key)

result = model.wv.most_similar(positive=['humanity'], negative = ['war'])
most_similar_key, similarity = result[0]  # look at the first match
print("hummanity-war = "+most_similar_key)



king+woman-man = expensive
romantic+funny = smart
boy+mature = 50-year-old
man+clever = convincing
woman-dumb = man
woman-smart = young
hummanity-war = annoying


Some of these vector maths are ammusing. Because movies often play into various stereotypes, the vector math reflects that. Kings are often adorned with great riches and jewlery, movie culture will often dypict women as hyper materialistic. Therefore the math works out that a female king is not a queen but rather expensive.
<br>Worse than that, woman-dumb = man. I suppose this is due to the 'dumb blond' supporting role type that is often portrayed in movies. 